# Midterm Project: ETL and Data Warehousing
Madelyn Khoury (mgk5ybb) and Tiara Allard (tia4qp)

DS 2002 Spring 2023

## Design and Strategy

We chose to model bank transactions as the core business process of our data warehouse, so we designed a database schema centered around bank transactions. To see the schema we designed for this project, please look at the ReadMe of our GitHub project. 

Our schema stores information about bank transactions, bank accounts and users involved in transactions, transaction dates, and transaction locations. We were unable to find a database/dataset with all this information, so instead we combined data from multiple different data sources. This had the added benefit of allowing us to meet the requirements for importing data from a number of sources.

We combined several dummy/randomly generated datasets to build a complete data warehouse. First, we got bank transaction and account information from a .csv file stored on our local filesystem. Then, we generated user data from an API and linked it to the accounts and transactions. Finally, we imported location information from the Northwind MySQL database to represent regions in which banking transactions might have occurred.

After processing the data and computing useful fields, we formatted it into our fact and dimension tables in the final data warehouse.

## Imports and Helper Functions

In [ ]:
import sys
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install pymysql
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install uszipcode

In [64]:
import datetime
import json
import mysql.connector
import os
import pandas as pd
import pymysql
import random
import requests
from sqlalchemy import create_engine
from uszipcode import SearchEngine, SimpleZipcode

In [65]:
def get_api_response(url, headers, params, response_type):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        # result = json.dumps(response.json(), sort_keys=True, indent=4)
        result = response.json()
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"
        
    return result

In [66]:
# this helper function is inspired by part of one of the provided files, 02-Python-MySQL.ipynb
def get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    dframe = None
    try:
        conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
        sqlEngine = create_engine(conn_str, pool_recycle=3600)
        connection = sqlEngine.connect()
        try:
            dframe = pd.read_sql(sql_query, connection);
        except:
            print("Sequel query was unsuccessful.")
        connection.close()
        return dframe
    except:
        print("Unable to connect to the MySQL database.")
    return None

In [47]:
# this code snippet is modified from: https://www.geeksforgeeks.org/python-program-to-calculate-age-in-year/ 
def calculate_age(birth_date):
    birth_date = datetime.datetime.strptime(birth_date, '%Y-%m-%d').date()
    today = datetime.date.today()
    try:
        birthday = birth_date.replace(year = today.year)
 
    # raised when birth date is February 29 but it's not a leap year
    except ValueError:
        birthday = birth_date.replace(year = today.year,
                  month = birth_date.month + 1, day = 1) # birth date becomes march 1st
 
    if birthday > today:
        return today.year - birth_date.year - 1
    else:
        return today.year - birth_date.year

In [64]:
def get_region(state_abbreviation):
    if state_abbreviation in {"WA", "OR", "CA", "ID", "MT", "NV", "UT", "CO", "WY", "AK"}:
        return "West"
    elif state_abbreviation in {"AZ", "NM", "TX", "OK"}:
        return "Southwest"
    elif state_abbreviation in {"ND", "SD", "NE", "KS", "MN", "IA", "MO", "WI", "IL", "MI", "OH"}:
        return "Midwest"
    elif state_abbreviation in {"ME", "NH", "MA", "CT", "RI", "VT", "NY", "PA", "DE", "MD", "NJ"}:
        return "Northeast"
    else:
        return "Southeast"

In [93]:
def get_zipcode(city, state):
    search = SearchEngine()
    results = search.by_city_and_state(city, state)
    if len(results) > 0:
        return results[0].zipcode
    else:
        return None

In [208]:
def get_day(date_timestamp):
    return date_timestamp.day

In [203]:
def get_month(date_timestamp):
    return date_timestamp.month_name()

In [212]:
def get_year(date_timestamp):
    return date_timestamp.year

In [43]:
def get_week_day(date_timestamp):
    return date_timestamp.day_name()

## Loading in Data
In this section, we will be loading in the data from three sources: an API, a local filesystem, and a relational database.

### Importing Data From Local File System

The core bank transaction information that we will use came from a dataset on Kaggle (https://www.kaggle.com/datasets/apoorvwatsky/bank-transaction-data). We downloaded the data in the form of a xlsx file and will import it from the local filesystem in order to be used in our data warehouse.

In [67]:
bank_info_path = os.path.join(os.getcwd(), 'bank.xlsx')
bank_info = pd.read_excel(bank_info_path)

In [68]:
bank_info

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


### Importing Data From API

We've chosen to use the `users` endpoint from random-data-api.com, which randomly generates data for a set of users. This will populate the Users table in our data warehouse.

In [69]:
size = 10 # only get info on 10 users for now
url = "https://random-data-api.com/api/v2/users"
querystring = {"size":size}
headers = None

# Get information from users API endpoint
users = get_api_response(url, headers, querystring, "dataframe")
users

,id,uid,password,first_name,last_name,username,email,avatar,gender,phone_number,...,address.zip_code,address.state,address.country,address.coordinates.lat,address.coordinates.lng,credit_card.cc_number,subscription.plan,subscription.status,subscription.payment_method,subscription.term
0,6827,42d5245f-80d9-42fa-a11f-bcbe8a1c442a,RCxboQgrAX,Felicitas,Luettgen,felicitas.luettgen,felicitas.luettgen@email.com,https://robohash.org/autvoluptasincidunt.png?s...,Polygender,+372 1-483-127-4186 x1831,...,13518,Rhode Island,United States,18.239511,-150.690530,4429-2049-6996-4868,Bronze,Pending,WeChat Pay,Annual
1,1163,74add38a-094c-4d2e-8951-00f5ea16481d,twWV0dvKsU,Eduardo,Windler,eduardo.windler,eduardo.windler@email.com,https://robohash.org/totamnatusvoluptas.png?si...,Polygender,+216 (306) 734-4322 x40416,...,44272,Alabama,United States,71.973495,103.374405,4441747296657,Professional,Idle,Debit card,Payment in advance
2,7451,9ef58a1d-5c29-4a6a-95ec-5f33a8b66f7c,RwF1HjVbNA,Denisha,Boyle,denisha.boyle,denisha.boyle@email.com,https://robohash.org/suntpariaturcorporis.png?...,Polygender,+86 1-271-491-4790,...,41598-6463,Washington,United States,63.770340,4.684313,4948-0820-7234-8829,Starter,Idle,Credit card,Annual
3,3369,278587f9-c003-471e-a0bf-f0091e006868,4MpF2vYKsc,Lala,Dicki,lala.dicki,lala.dicki@email.com,https://robohash.org/perspiciatisquibusdamlibe...,Genderfluid,+31 973.512.9150 x758,...,84789,Nevada,United States,18.696701,21.541486,4684-9539-2448-0945,Bronze,Blocked,Cash,Full subscription
4,5486,3e7d6010-425f-4004-be7b-5dc8d7c5f6db,D7QlUp5cWq,Tiesha,Wilderman,tiesha.wilderman,tiesha.wilderman@email.com,https://robohash.org/doloresconsecteturest.png...,Genderfluid,+266 342.422.0453 x40969,...,59485-9906,Arizona,United States,-3.988568,-70.615170,6771-8941-8981-6846,Starter,Blocked,Bitcoins,Full subscription
5,1674,7e157461-fcb9-4c54-a92f-740dfbf332d1,r0m9IhtNTW,Kris,Stoltenberg,kris.stoltenberg,kris.stoltenberg@email.com,https://robohash.org/laboriosamfacilislaborum....,Female,+1-268 1-794-603-4146,...,69438,Iowa,United States,-32.445460,-174.569648,4284262321132,Bronze,Idle,Cash,Monthly
6,8753,956e4bf4-aa5d-42e6-a9f1-67a4d7d0e885,R9dFDvnL2E,Carmelita,Schimmel,carmelita.schimmel,carmelita.schimmel@email.com,https://robohash.org/veliteaesse.png?size=300x...,Polygender,+252 (910) 003-2423 x716,...,79257,Illinois,United States,54.473171,117.221683,6771-8989-0613-1322,Basic,Blocked,Money transfer,Full subscription
7,8349,fe7fa776-c5cb-4408-8320-c5a86bdf73c6,S6h798VAvH,Lorina,Lowe,lorina.lowe,lorina.lowe@email.com,https://robohash.org/aliquameavoluptas.png?siz...,Agender,+855 (693) 325-6994,...,19452,Montana,United States,18.813268,134.237430,6771-8925-9599-1583,Bronze,Pending,Credit card,Monthly
8,9829,13f4c0cc-a4df-4758-8740-5c227ae4d98c,niMAP7jQUF,Stacia,Balistreri,stacia.balistreri,stacia.balistreri@email.com,https://robohash.org/commodirerumet.png?size=3...,Genderfluid,+231 841-918-6671,...,45732-1801,Ohio,United States,84.105393,-70.835933,4842295670013,Basic,Pending,WeChat Pay,Annual
9,6448,24dff196-81a0-478f-a0a3-17f5c2d17e4f,OAX5Ip0rWZ,Luz,Johnson,luz.johnson,luz.johnson@email.com,https://robohash.org/etetipsum.png?size=300x30...,Female,+886 (655) 079-3531 x32365,...,22256,Maine,United States,88.449839,-41.378784,4433290839173,Starter,Pending,WeChat Pay,Monthly


### Importing Data From Relational Database

To get location data that could represent locations in which bank transactions were completed, we've decided to import information about the shipping location of orders from the `orders` table in the Northwind database. In our data warehouse, this information will represent the location in which a customer instigated a banking transaction; perhaps it could represent the location of physical branches of the bank.

In [70]:
# define variables to set up connection to mySQL database
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "northwind"

First we must get the location-related data from the `orders` table.

In [71]:
sql_query = """
    SELECT ship_address, ship_city, ship_state_province, ship_zip_postal_code, ship_country_region from orders;
"""

In [72]:
locations_info = get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query)

## Transforming/Cleaning Up the Data
In this section, we will be transforming, cleaning, and doing transformations on the data, as well as separating it into several tables that we can easily import into our data warehouse.

### Transforming the Location Data

We got location info from the Northwind database, but we must remove duplicate values so that we have a table of unique locations.

In [73]:
locations_info = locations_info.drop_duplicates()

It appears that the Northwind database didn't store actual zip codes, but instead put 99999 in for every row. So, we will fill in the table with the correct zip code for each city listed. Additionally, we will add another column to the table which will identify the region of the United States that the location is in.

In [74]:
locations_info["zipcode"] = locations_info.apply(lambda row: get_zipcode(row["ship_city"], row["ship_state_province"]), axis=1)
locations_info.drop(["ship_zip_postal_code"], axis = 1, inplace = True)

In [75]:
locations_info["region"] = locations_info.apply(lambda row: get_region(row["ship_state_province"]), axis=1)
locations_info

,ship_address,ship_city,ship_state_province,ship_country_region,zipcode,region
0,789 27th Street,Las Vegas,NV,USA,89101,West
1,123 4th Street,New York,NY,USA,10001,Northeast
2,123 12th Street,Las Vegas,NV,USA,89101,West
3,123 8th Street,Portland,OR,USA,97201,West
5,789 29th Street,Denver,CO,USA,80202,West
6,123 3rd Street,Los Angelas,CA,USA,90001,West
7,123 6th Street,Milwaukee,WI,USA,53202,Midwest
8,789 28th Street,Memphis,TN,USA,38103,Southeast
10,123 10th Street,Chicago,IL,USA,60601,Midwest
11,123 7th Street,Boise,ID,USA,83702,West


### Transforming the User Data

To transform the user data, all we had to do was drop some columns of the dataframe. As we can see by looking at the columns of users, there is a lot of superfluous information.

In [76]:
users.columns

Index(['id', 'uid', 'password', 'first_name', 'last_name', 'username', 'email',
       'avatar', 'gender', 'phone_number', 'social_insurance_number',
       'date_of_birth', 'employment.title', 'employment.key_skill',
       'address.city', 'address.street_name', 'address.street_address',
       'address.zip_code', 'address.state', 'address.country',
       'address.coordinates.lat', 'address.coordinates.lng',
       'credit_card.cc_number', 'subscription.plan', 'subscription.status',
       'subscription.payment_method', 'subscription.term'],
      dtype='object')

In [77]:
users.drop(['employment.title', 'employment.key_skill', 'uid','avatar', 'social_insurance_number', 'subscription.plan', 'subscription.payment_method', 'subscription.status', 'subscription.term', 'address.city', 'address.street_name', 'address.street_address', 'address.zip_code', 'address.state', 'address.country', 'address.coordinates.lat', 'address.coordinates.lng'], axis = 1, inplace = True)

We will also calculate the age of each user, that way we have calculations stored in our OLAP database and don't have to compute them on the fly.

In [78]:
users["age"] = users.apply(lambda row: calculate_age(row["date_of_birth"]), axis=1)

In [79]:
users

,id,password,first_name,last_name,username,email,gender,phone_number,date_of_birth,credit_card.cc_number,age
0,6827,RCxboQgrAX,Felicitas,Luettgen,felicitas.luettgen,felicitas.luettgen@email.com,Polygender,+372 1-483-127-4186 x1831,1968-08-19,4429-2049-6996-4868,54
1,1163,twWV0dvKsU,Eduardo,Windler,eduardo.windler,eduardo.windler@email.com,Polygender,+216 (306) 734-4322 x40416,1958-07-06,4441747296657,64
2,7451,RwF1HjVbNA,Denisha,Boyle,denisha.boyle,denisha.boyle@email.com,Polygender,+86 1-271-491-4790,1976-01-10,4948-0820-7234-8829,47
3,3369,4MpF2vYKsc,Lala,Dicki,lala.dicki,lala.dicki@email.com,Genderfluid,+31 973.512.9150 x758,1979-11-20,4684-9539-2448-0945,43
4,5486,D7QlUp5cWq,Tiesha,Wilderman,tiesha.wilderman,tiesha.wilderman@email.com,Genderfluid,+266 342.422.0453 x40969,2003-09-24,6771-8941-8981-6846,19
5,1674,r0m9IhtNTW,Kris,Stoltenberg,kris.stoltenberg,kris.stoltenberg@email.com,Female,+1-268 1-794-603-4146,1989-06-30,4284262321132,33
6,8753,R9dFDvnL2E,Carmelita,Schimmel,carmelita.schimmel,carmelita.schimmel@email.com,Polygender,+252 (910) 003-2423 x716,1993-03-19,6771-8989-0613-1322,30
7,8349,S6h798VAvH,Lorina,Lowe,lorina.lowe,lorina.lowe@email.com,Agender,+855 (693) 325-6994,1985-06-28,6771-8925-9599-1583,37
8,9829,niMAP7jQUF,Stacia,Balistreri,stacia.balistreri,stacia.balistreri@email.com,Genderfluid,+231 841-918-6671,1988-05-20,4842295670013,34
9,6448,OAX5Ip0rWZ,Luz,Johnson,luz.johnson,luz.johnson@email.com,Female,+886 (655) 079-3531 x32365,1989-02-16,4433290839173,34


### Transforming the Account Information

Reviewing the bank_info table we made, we can see that it has columns relating not just to a single transaction, but also to the date and account associated with the transaction.

In [80]:
bank_info.columns

Index(['Account No', 'DATE', 'TRANSACTION DETAILS', 'CHQ.NO.', 'VALUE DATE',
       'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT', '.'],
      dtype='object')

We will separate this data into three tables: a Transactions fact table, an Accounts dimension, and a Date dimension. 
The bank transaction fact table can store the new balance of an account after a transaction is completed, but we want the bank account table to store current info about each account-- information which is independent of any one transaction. So, we will take the most recent balance for each account and create a "Current Balance" field in the Accounts table.  Since the rows of the spreadsheet were sorted in order of transaction date, then the most recent balance for each account will be the balance in the last-occurring transaction.

In [81]:
account_info = bank_info[["Account No", "BALANCE AMT"]]
account_info

,Account No,BALANCE AMT
0,409000611074',1.000000e+06
1,409000611074',2.000000e+06
2,409000611074',2.500000e+06
3,409000611074',5.500000e+06
4,409000611074',6.000000e+06
...,...,...
116196,409000362497',-1.901902e+09
116197,409000362497',-1.901602e+09
116198,409000362497',-1.901302e+09
116199,409000362497',-1.901412e+09


In [82]:
account_info = account_info.drop_duplicates(subset=["Account No"], keep="last") # keep only the last record for each account
account_info = account_info.reset_index(drop=True) # I'm also going to reset the indices to start from 0

Our schema design includes another field in the accounts table: the ID of the customer who holds this account. We will randomly select users from our users table to act as the "holders" of these accounts. We will also randomly select the type of each account from a list of options.

In [83]:
account_info["User ID"] = account_info.apply(lambda row: random.randint(0, users.shape[0]-1), axis=1)

In [84]:
bank_account_types = ["Checking", "Savings", "Money market (MMA)", "Certificate of deposit (CD)"]
account_info["Account Type"] = account_info.apply(lambda row: bank_account_types[random.randint(0, len(bank_account_types)-1)], axis=1)

In [85]:
account_info

,Account No,BALANCE AMT,User ID,Account Type
0,409000611074',4.622000e+05,4,Savings
1,409000493201',7.435833e+05,7,Money market (MMA)
2,409000425051',-3.567348e+08,5,Money market (MMA)
3,409000405747',-5.482675e+08,6,Certificate of deposit (CD)
4,409000438611',-5.479193e+08,1,Savings
5,409000493210',-5.463146e+08,1,Certificate of deposit (CD)
6,409000438620',-5.399631e+08,6,Checking
7,1196711',-1.586916e+09,9,Savings
8,1196428',-1.687234e+09,4,Money market (MMA)
9,409000362497',-1.901417e+09,0,Checking


### Transforming the Date Information

The other aspect of transactions that we want to put in its own dimension table is the date of each transaction. We will generate a date entry for each date that occurs in the table of transaction information, then link the two tables together.

In [86]:
# get a list of unique dates from the table of transaction info
date_info = bank_info.drop_duplicates(subset=["DATE"])
date_info = date_info.reset_index(drop=True) # reset the indices to start from 0
date_info.drop([ 'TRANSACTION DETAILS', 'CHQ.NO.','VALUE DATE', 'WITHDRAWAL AMT', 'DEPOSIT AMT', 'BALANCE AMT', '.'], axis = 1, inplace = True)
date_info

,Account No,DATE
0,409000611074',2017-06-29
1,409000611074',2017-07-05
2,409000611074',2017-07-18
3,409000611074',2017-08-01
4,409000611074',2017-08-16
...,...,...
1289,409000362497',2017-12-25
1290,409000362497',2018-04-02
1291,409000362497',2018-04-29
1292,409000362497',2018-05-12


In [87]:
date_info["Day"] = date_info.apply(lambda row: get_day(row["DATE"]), axis=1)

In [88]:
date_info["Month"] = date_info.apply(lambda row: get_month(row["DATE"]), axis=1)

In [89]:
date_info["Year"] = date_info.apply(lambda row: get_year(row["DATE"]), axis=1)

In [90]:
date_info["Day of the Week"] = date_info.apply(lambda row: get_week_day(row["DATE"]), axis=1)

In [91]:
date_info

,Account No,DATE,Day,Month,Year,Day of the Week
0,409000611074',2017-06-29,29,June,2017,Thursday
1,409000611074',2017-07-05,5,July,2017,Wednesday
2,409000611074',2017-07-18,18,July,2017,Tuesday
3,409000611074',2017-08-01,1,August,2017,Tuesday
4,409000611074',2017-08-16,16,August,2017,Wednesday
...,...,...,...,...,...,...
1289,409000362497',2017-12-25,25,December,2017,Monday
1290,409000362497',2018-04-02,2,April,2018,Monday
1291,409000362497',2018-04-29,29,April,2018,Sunday
1292,409000362497',2018-05-12,12,May,2018,Saturday


In [92]:
bank_info

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


In [93]:
type(date_info["DATE"][0])

pandas._libs.tslibs.timestamps.Timestamp

### Transforming the Transaction Data

Finally, we can clean up the transaction data!

In [94]:
bank_info

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


In [95]:
bank_info.drop(['CHQ.NO.','VALUE DATE', 'BALANCE AMT', '.'], axis = 1, inplace = True)
bank_info

,Account No,DATE,TRANSACTION DETAILS,WITHDRAWAL AMT,DEPOSIT AMT
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,1000000.0
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,1000000.0
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,500000.0
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,3000000.0
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,500000.0
...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,117934.30,NaN
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,300000.0
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,300000.0
116199,409000362497',2019-03-05,IMPS 05-03-20194C,109868.65,NaN


## Creating and Populating the Data Warehouse
In this section, we will create a new SQL database with all the tables specified in our schema. We'll then populate it with the data tables that we created earlier.

## Executing SQL Queries
Finally, in this section, we will execute several SQL queries that aggregate data from at least three of the data tables in our data warehouse.

In [98]:
sql_query = "SELECT loc_id from locations;"
db_name = "banks"
locations_info = get_mysql_dataframe(user_id, pwd, host_name, db_name, sql_query)
locations_info

,loc_id
